In [1]:
import tensorflow as tf
import numpy as np

# 设置numpy浮点数的长度
np.set_printoptions(precision=3, suppress=True)

### 看一个例子，理解SparseCategoricalCrossentropy()是如何计算的

+ 设置from_logits=True，表示输入样本的是得分值而不是概率值，函数会自动使用softmax将其转换成概率值
+ 修改reduction参数，也可以得到每个样本的交叉熵损失, None表示返回每个样本的交叉熵，SUM表示返回交叉熵之和, 默认返回所有交叉熵的平均值

In [2]:
scc_auto = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

scc_none = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
            reduction=tf.keras.losses.Reduction.NONE)

scc_sum = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
            reduction=tf.keras.losses.Reduction.SUM)

+ 计算这组数据的交叉熵损失，假设共有3个类别(0, 1, 2)，3个样本
+ 每个样本在对应类别上的预测值都是[1, 2, 3]
+ API计算的结果是1.407606(平均值)

In [3]:
y_true_1 = np.array([0, 1, 2])
y_pred_1 = np.array([[6., 2., 3.], [2., 1., 4.], [3., 2., 1.]])

In [4]:
y_true_2 = np.array([[0], [1], [2]])
y_pred_2 = np.array([[6., 2., 3.], [2., 1., 4.], [3., 2., 1.]])

scc对y_true_1和y_true_2的计算结果是相同的，尽管两者数据维度不相同

In [5]:
scc_auto(y_true_1, y_pred_1).numpy() # 平均值

1.881111979484558

In [6]:
scc_none(y_true_1, y_pred_1).numpy() # 每个样本的损失值

array([0.066, 3.17 , 2.408])

In [7]:
scc_sum(y_true_1, y_pred_1).numpy() # 损失值的和

5.643335819244385

In [8]:
scc_auto(y_true_2, y_pred_2).numpy()

1.881111979484558

In [9]:
scc_none(y_true_2, y_pred_2).numpy()

array([0.066, 3.17 , 2.408])

In [10]:
scc_sum(y_true_2, y_pred_2).numpy()

5.643335819244385

下面手动模拟一遍，首先需要把预测值1，2，3通过softmax()函数转换成概率值，可以使用keras提供的softmax()函数先将预测值转换成概率值

+ 注意输入参数的值必须是二维的，且必须是tensor对象，每一行表示一个样本在各个类别上的得分值
+ 计算公式相当于exp(x) / tf.reduce_sum(exp(x))

In [11]:
t = tf.keras.activations.softmax(tf.constant(y_pred_1))
t

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[0.936, 0.017, 0.047],
       [0.114, 0.042, 0.844],
       [0.665, 0.245, 0.09 ]])>

+ 得到概率值后，根据每个样本的正确类别计算交叉熵损失
+ 还需要将标签值转换成独热编码的形式，使用tf.one_hot函数

In [12]:
y_true_one_hot = tf.one_hot(indices=y_true_1, depth=3, dtype=tf.float64)
y_true_one_hot

<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])>

In [13]:
# 计算交叉熵，每行是一个样本，注意加上负号
# 因为独热编码只有一个元素不为0，因此每一行只有对应正确类别才有损失值
loss = -y_true_one_hot * tf.math.log(t)

+ 最后对每一行求和即可，可以看到与API计算的结果一致

In [14]:
tf.math.reduce_sum(loss, axis=1)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([0.066, 3.17 , 2.408])>